In [2]:
import pandas as pd
import numpy as np
import os

import yaml
#import xml.etree.ElementTree as ET
from xml_to_dict import XMLtoDict
parser = XMLtoDict()

import jellyfish

# Reading data from SysRev from the Title and Abstract screening

In [2]:
!pwd

/home/ls/Documents/scr/phd/north_sea_review/data_pipeline


In [4]:

#sysrev = pd.read_csv('sysrev/2021_08_29_Articles_P37916_0820_A6587.csv')
sysrev = pd.read_csv('sysrev/2021_11_05_Articles_P37916_1105_A7440.csv')

# Merging Sysrev data with metadata available from the search engine databases

## CSV Databases

### DATA

In [5]:
Core = pd.read_csv('search_engine_data/Metadata_full_articles_Core.csv')
Scopus = pd.read_csv('search_engine_data/Metadata_full_articles_Scopus.csv')
WoS = pd.read_csv('search_engine_data/Metadata_full_articles_WoS.csv')

search_engine_data = {}
for i in ('Core','Scopus','WoS'):
    search_engine_data[i] = locals()[i]

/sw/buster-x64/anaconda3-2020.02/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### DATA Structure

In [7]:
with open('csv_headers.yml', 'r') as file:
        data = file.read()
csv_headers = yaml.safe_load(data)

### Creating unified data frame

### MAJOR ISSUE WITH SCOPUS. SOME KIND OF CHARACTER ERROR AFTER ENTRY 8838

In [8]:
csv_databases = pd.DataFrame({}, columns=list(csv_headers))
for db in search_engine_data:
    #if db != 'WoS': continue
    unified_df = pd.DataFrame({})
    
    for header in csv_headers:
        #print(header)
        #print([csv_headers[header][db]])
        key = csv_headers[header][db]
/data_pipeline/
Name
Last Modified

1

import pandas as pd

2

import numpy as np

3

import os
        if key is None:
            unified_df[header] = None
        else:
            temp = search_engine_data[db][key]
            unified_df[header] = temp
    unified_df['Database'] = db
    print(db)

    # temp fix for something that is fucked up in the scopus database
    if db == 'Scopus':
        # THIS IS A MAJOR ISSUE FIX IT!
        csv_databases = csv_databases.append(unified_df[:8837])
    else:
        csv_databases = csv_databases.append(unified_df)

csv_databases['metadata filetype'] = 'csv'    

#csv_databases[csv_databases['Database'] == 'Scopus'] = csv_databases[csv_databases['Database'] == 'Scopus'][:8838]

Core
Scopus
WoS


## XML

### DATA

In [9]:
GoogleScholar = 'search_engine_data/Google_Scholar_1945-2020.xml'
BASE = 'search_engine_data/BASE_1945_2020_final.xml'
PubMed = 'search_engine_data/1945-2020_PubMed_clean_2.xml'

xml_engine_data = {}
for i in ('GoogleScholar','BASE','PubMed'):
    with open(locals()[i], 'r') as file:
        data = file.read()
    xml_engine_data[i] = parser.parse(data)['xml']['records']['record']

### DATA Structure

In [10]:
with open('xml_headers.yml', 'r') as file:
        data = file.read()
xml_headers = yaml.safe_load(data)

In [13]:
# create empty array with the right headers and proper length
xml_databases = pd.DataFrame({}, columns=list(xml_headers))
for db in xml_engine_data:
    #print(f'db: {db}')
    #if ii != 0: continue
    data = xml_engine_data[db]
    unified_df = pd.DataFrame('', index=np.arange(len(data)), columns=list(xml_headers))
    for ii,item in enumerate(data):
        #print(f'item number: {ii}')
        for header in xml_headers:
            #print(f'header: {header}')
            stack = xml_headers[header][db]
            if stack is None: continue

            temp = item
            #print(f'stacked temp: {temp}')
            #print(f'stack: {stack}')
            for kk,element in enumerate(stack):
                try:
                    temp = temp[element]
                    #print(f'temp: {temp}')
                    if kk == len(stack)-1:
                        #print(f'temp: {temp}')
                        unified_df[header][ii] = temp
                except:
                        unified_df[header][ii] = None
    unified_df['Database'] = db
    xml_databases = xml_databases.append(unified_df)
xml_databases['metadata filetype'] = 'xml'

## merging csv and xml

In [14]:
metadata_df = pd.concat([csv_databases,xml_databases])

## Cleaning metadata 

In [15]:
# reindexing to have a full length consistent index
metadata_df['db index'] = metadata_df.index
metadata_df['index'] = np.arange(metadata_df.shape[0])
metadata_df = metadata_df.set_index('index')

In [16]:
# dropping nan and None values
def fill_npnan(item):
    if type(item) is float:
        if np.isnan(item):
            return ''

metadata_df = metadata_df.fillna('')
metadata_df.apply(fill_npnan)

Authors              None
Affiliations         None
Title                None
Abstract             None
Year                 None
Type                 None
Journal              None
Doi                  None
Link                 None
Database             None
metadata filetype    None
db index             None
dtype: object

In [17]:
def clean_authors(item):
    if type(item) is str:
        if '; ' in item:
            item = item.split('; ')
        elif True:
            pass
            # there are severl other issues.
            # ie in PubMed authors are seperated by ',' however first and last name are also seperated by ','
            # we ignore all this for the time being            
        else:
            item = [item]
    return item

metadata_df.loc[:,'Authors'] = metadata_df.loc[:,'Authors'].apply(clean_authors)

In [18]:
for item in set(metadata_df.Database):
    print(item)
    print(metadata_df.loc[metadata_df.Database == item,'Title'])

WoS
index
10344    Long-term intercomparison of two pCO(2) instru...
10345    With a little help from my friends: physiologi...
10346    Cross-continental analysis of coastal biodiver...
10347    Bigger juveniles and smaller adults: Changes i...
10348    Toxicity and biomarkers of micro-plastic in aq...
                               ...                        
18500    ECOLOGICAL STUDIES ON MACROINVERTEBRATE POPULA...
18501             BRITAIN EXPLOITS ITS NORTH SEA GAS FINDS
18502           RECENT CHANGES IN NORTH SEA PLAICE FISHERY
18503    EEL FISHING IN STATES OF LOWER SAXONY AND BREM...
18504    EEL FISHING IN STATE SCHLESWIG-HOLSTEIN . NORT...
Name: Title, Length: 8161, dtype: object
GoogleScholar
index
18505    Rapana venosa (Gastropoda: Muricidae): a new i...
18506    A new and alien species of “oyster leech”(Plat...
18507    European native oysters and associated species...
18508    Invasive Alien Species in the Belgian part of ...
18509    Checklist for aquatic alien species

In [19]:
set([type(item) for item in metadata_df.Authors])
# there are still some issues.

{list, str}

## Matching metadata from from search engine databases with those form SysRev and merging it

In [20]:
# creating empty "merged dataframe" with the right headers
matching_header = ['match quality', 'meta_df_index']

merged_high_level_header = len(sysrev.columns)*['sysrev'] + len(metadata_df.columns)*['metadata'] + len(matching_header)*['matching']
merged_low_level_header = list(sysrev.columns) + list(metadata_df.columns) + matching_header
merged_columns = pd.MultiIndex.from_arrays([merged_high_level_header,merged_low_level_header])

merged_df = pd.DataFrame(index=np.arange(len(sysrev)),columns=merged_columns)
merged_df

sysrev                                             \
     Article ID Article URL Title Journal Authors Abstract   
0           NaN         NaN   NaN     NaN     NaN      NaN   
1           NaN         NaN   NaN     NaN     NaN      NaN   
2           NaN         NaN   NaN     NaN     NaN      NaN   
3           NaN         NaN   NaN     NaN     NaN      NaN   
4           NaN         NaN   NaN     NaN     NaN      NaN   
...         ...         ...   ...     ...     ...      ...   
7435        NaN         NaN   NaN     NaN     NaN      NaN   
7436        NaN         NaN   NaN     NaN     NaN      NaN   
7437        NaN         NaN   NaN     NaN     NaN      NaN   
7438        NaN         NaN   NaN     NaN     NaN      NaN   
7439        NaN         NaN   NaN     NaN     NaN      NaN   

                            metadata                     ...            \
     predict(47459:Include)  Authors Affiliations Title  ... Year Type   
0                       NaN      NaN          NaN   NaN  ...  NaN  NaN   
1                       NaN      NaN          NaN   NaN  ...  NaN  NaN   
2                       NaN      NaN          NaN   NaN  ...  NaN  NaN   
3                       NaN      NaN          NaN   NaN  ...  NaN  NaN   
4                       NaN      NaN          NaN   NaN  ...  NaN  NaN   
...                     ...      ...          ...   ...  ...  ...  ...   
7435                    NaN      NaN          NaN   NaN  ...  NaN  NaN   
7436                    NaN      NaN          NaN   NaN  ...  NaN  NaN   
7437                    NaN      NaN          NaN   NaN  ...  NaN  NaN   
7438                    NaN      NaN          NaN   NaN  ...  NaN  NaN   
7439                    NaN      NaN          NaN   NaN  ...  NaN  NaN   

                                                                matching  \
     Journal  Doi Link Database metadata filetype db index match quality   
0        NaN  NaN  NaN      NaN               NaN      NaN           NaN   
1        NaN  NaN  NaN      NaN               NaN      NaN           NaN   
2        NaN  NaN  NaN      NaN               NaN      NaN           NaN   
3        NaN  NaN  NaN      NaN               NaN      NaN           NaN   
4        NaN  NaN  NaN      NaN               NaN      NaN           NaN   
...      ...  ...  ...      ...               ...      ...           ...   
7435     NaN  NaN  NaN      NaN               NaN      NaN           NaN   
7436     NaN  NaN  NaN      NaN               NaN      NaN           NaN   
7437     NaN  NaN  NaN      NaN               NaN      NaN           NaN   
7438     NaN  NaN  NaN      NaN               NaN      NaN           NaN   
7439     NaN  NaN  NaN      NaN               NaN      NaN           NaN   

                    
     meta_df_index  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN  
...            ...  
7435           NaN  
7436           NaN  
7437           NaN  
7438           NaN  
7439           NaN  

[7440 rows x 21 columns]

In [26]:
for jj,sysrev_title in enumerate(sysrev['Title']):
    #if jj > 3: break
    if jj%1000 == 0: print(f'{jj/len(sysrev)*100}%')
    
    # MATCHING
    
    #print(sysrev_title)
    best_match = {'match': 0, 'db': None,
                  'meta_index': None, 'title': None}
    for ii,meta_title in enumerate(metadata_df['Title']):
        database = metadata_df.loc[ii,'Database']
        #if ii%1000 == 0: print(ii)
        try:
            match = jellyfish.jaro_distance(sysrev_title,meta_title)
        except TypeError:
            pass
            #print(f'sysrev_title: {sysrev_title}\nmeta_title: {meta_title}')

           
        if sysrev_title == meta_title:
            best_match['match'] = match
            best_match['db'] = database
            best_match['meta_index'] = ii
            best_match['title'] = meta_title
            #print('ideal match found (equal)',best_match['match'])
            #print(f'sysrev_title: {sysrev_title}\nmeta_title: {meta_title}\n')
            break
                
        
        if match == 1:
            best_match['match'] = match
            best_match['db'] = database
            best_match['meta_index'] = ii
            best_match['title'] = meta_title
            
            #print('ideal match found (match == 1)',best_match['match'])
            #print(f'sysrev_title: {sysrev_title}\nmeta_title: {meta_title}\n')
            break 
            
        if match > best_match['match']:
            best_match['match'] = match
            best_match['db'] = database
            best_match['meta_index'] = ii
            best_match['title'] = meta_title
            
    #print('non-ideal match found',best_match['match'])
    #print(f'sysrev_title: {sysrev_title}\nmeta_title: {meta_title}\n')
    # MERGING
    
    merged_df.loc[jj,'sysrev'] = list(sysrev.loc[jj])
    merged_df.loc[jj,'metadata'] = list(metadata_df.loc[best_match['meta_index']])
    merged_df.loc[jj,'matching'] = [best_match['match'],best_match['meta_index']]
    
                
            
merged_df.to_pickle('merged_df.pkl')
merged_df

13.440860215053762%
26.881720430107524%
40.32258064516129%
53.76344086021505%
67.20430107526882%
80.64516129032258%
94.08602150537635%


sysrev                                               \
     Article ID                                  Article URL   
0       9362819   https://sysrev.com/p/37916/article/9362819   
1       9362821   https://sysrev.com/p/37916/article/9362821   
2       9362825   https://sysrev.com/p/37916/article/9362825   
3       9362827   https://sysrev.com/p/37916/article/9362827   
4       9362828   https://sysrev.com/p/37916/article/9362828   
...         ...                                          ...   
7435   12028517  https://sysrev.com/p/37916/article/12028517   
7436   12028518  https://sysrev.com/p/37916/article/12028518   
7437   12028523  https://sysrev.com/p/37916/article/12028523   
7438   12028525  https://sysrev.com/p/37916/article/12028525   
7439   12028526  https://sysrev.com/p/37916/article/12028526   

                                                         \
                                                  Title   
0       Contemporary monitoring of storm surge activity   
1     Reconstruction of Atlantic herring (Clupea har...   
2     Modelling and predicting habitats for the neob...   
3     Multi-residue quantification and screening of ...   
4     Molecular analysis of Ulva compressa (Chloroph...   
...                                                 ...   
7435  The lifespan of ingested plastic particles in ...   
7436  Overview of the biological effects of lost and...   
7437               Ecological history of the Wadden Sea   
7438     Plastic ingestion in the North Atlantic fulmar   
7439  Plastic particle pollution: Accumulation by pr...   

                                                         \
                                                Journal   
0     Progress in Physical Geography-Earth and Envir...   
1                                    Fish and Fisheries   
2                   Estuarine Coastal and Shelf Science   
3                             Marine Pollution Bulletin   
4                         European Journal of Phycology   
...                                                 ...   
7435                          Marine Pollution Bulletin   
7436                          Marine Pollution Bulletin   
7437                          Helgoland Marine Research   
7438                          Marine Pollution Bulletin   
7439                          Marine Pollution Bulletin   

                                                         \
                                                Authors   
0                                 Ji, Tao; Li, Guosheng   
1     Estrella-Martinez, Juan; Schone, Bernd R.; Thu...   
2     Schwemmer, Philipp; Adler, Sven; Enners, Leoni...   
3     Vanryckeghem, Francis; Huysman, Steve; Van Lan...   
4             Steinhagen, S.; Weinberger, F.; Karez, R.   
...                                                 ...   
7435                           Ryan, P. G.; Jackson, S.   
7436                                       Laist, D. W.   
7437                            Lotze, H. K.; Reise, K.   
7438                                van Franeker, J. A.   
7439                                     Furness, R. W.   

                                                         \
                                               Abstract   
0     There is growing interest in storm surge activ...   
1     Understanding the recruitment variability of t...   
2     Neobiotic species can have profound impacts on...   
3     Knowledge about the occurrence of emerging org...   
4     To resolve historical misinterpretations of sp...   
...                                                 ...   
7435  The assimilation efficiencies of fledgling Whi...   
7436  In the past thirty years, the use of plastics ...   
7437                                                NaN   
7438  Fulmars found dead on the Dutch coast, and ful...   
7439  Plastic particles were found in the gizzards o...   

                             \
     predict(47459:Include)   
0                     0.023   
1          

0.0%
13.440860215053762%
26.881720430107524%
40.32258064516129%
53.76344086021505%
67.20430107526882%
80.64516129032258%
94.08602150537635%


sysrev                                               \
     Article ID                                  Article URL   
0       9362819   https://sysrev.com/p/37916/article/9362819   
1       9362821   https://sysrev.com/p/37916/article/9362821   
2       9362825   https://sysrev.com/p/37916/article/9362825   
3       9362827   https://sysrev.com/p/37916/article/9362827   
4       9362828   https://sysrev.com/p/37916/article/9362828   
...         ...                                          ...   
7435   12028517  https://sysrev.com/p/37916/article/12028517   
7436   12028518  https://sysrev.com/p/37916/article/12028518   
7437   12028523  https://sysrev.com/p/37916/article/12028523   
7438   12028525  https://sysrev.com/p/37916/article/12028525   
7439   12028526  https://sysrev.com/p/37916/article/12028526   

                                                         \
                                                  Title   
0       Contemporary monitoring of storm surge activity   
1     Reconstruction of Atlantic herring (Clupea har...   
2     Modelling and predicting habitats for the neob...   
3     Multi-residue quantification and screening of ...   
4     Molecular analysis of Ulva compressa (Chloroph...   
...                                                 ...   
7435  The lifespan of ingested plastic particles in ...   
7436  Overview of the biological effects of lost and...   
7437               Ecological history of the Wadden Sea   
7438     Plastic ingestion in the North Atlantic fulmar   
7439  Plastic particle pollution: Accumulation by pr...   

                                                         \
                                                Journal   
0     Progress in Physical Geography-Earth and Envir...   
1                                    Fish and Fisheries   
2                   Estuarine Coastal and Shelf Science   
3                             Marine Pollution Bulletin   
4                         European Journal of Phycology   
...                                                 ...   
7435                          Marine Pollution Bulletin   
7436                          Marine Pollution Bulletin   
7437                          Helgoland Marine Research   
7438                          Marine Pollution Bulletin   
7439                          Marine Pollution Bulletin   

                                                         \
                                                Authors   
0                                 Ji, Tao; Li, Guosheng   
1     Estrella-Martinez, Juan; Schone, Bernd R.; Thu...   
2     Schwemmer, Philipp; Adler, Sven; Enners, Leoni...   
3     Vanryckeghem, Francis; Huysman, Steve; Van Lan...   
4             Steinhagen, S.; Weinberger, F.; Karez, R.   
...                                                 ...   
7435                           Ryan, P. G.; Jackson, S.   
7436                                       Laist, D. W.   
7437                            Lotze, H. K.; Reise, K.   
7438                                van Franeker, J. A.   
7439                                     Furness, R. W.   

                                                         \
                                               Abstract   
0     There is growing interest in storm surge activ...   
1     Understanding the recruitment variability of t...   
2     Neobiotic species can have profound impacts on...   
3     Knowledge about the occurrence of emerging org...   
4     To resolve historical misinterpretations of sp...   
...                                                 ...   
7435  The assimilation efficiencies of fledgling Whi...   
7436  In the past thirty years, the use of plastics ...   
7437                                                NaN   
7438  Fulmars found dead on the Dutch coast, and ful...   
7439  Plastic particles were found in the gizzards o...   

                             \
     predict(47459:Include)   
0                     0.023   
1          

In [27]:
merged_df = pd.read_pickle('merged_df.pkl')

# Extending metadata by Crossref Lookup with data from sysrev (not search engine data - this one is not validated!) - trying to fetch dois where possible

In [28]:
from habanero import Crossref
import time

def flatten(list_of_strings):
    if type(list_of_strings) == list:
        string = ""
        for item in list_of_strings:
            string += " "+item
        return string
    elif type(list_of_strings) == str:
        return list_of_strings
    else:
        print(f"From flatten: {list_of_strings}")
        return ''
    
def fetch_doi_from_crossref(item):
    """ link titles with dois """
    cr = Crossref()

    # goes thru all the papers and checks via crossref
    query = '"' + item["title"] + '"'\
          + " " + flatten(item["authors"])
    print(query)
    
    server_reached = False
    while server_reached == False:
        try:
            query_result = cr.works(query=query, limit=1)
            server_reached = True
        except:
            # HTTPError (Service Unavailable)
            print("CrossRef server unavailable. Retry in 5 seconds")
            time.sleep(5)

    try:
        title = query_result['message']['items'][0]['title'][0]
    except KeyError:
        title = 'None'

    doi = query_result['message']['items'][0]['DOI']
    
    # test matching
    matching_quali = jellyfish.jaro_distance(item['title'].lower().replace('  ',' '),title.lower().replace(' ',''))
        
    if matching_quali < 0.9:
        doi = ''
        title = ''
    
    return doi, title

In [29]:
merged_df['crossref','lookup'] = False
merged_df['crossref','Title'] = ''

In [30]:
for ii in range(len(merged_df)):
    if merged_df.loc[ii,('metadata','Doi')] == '':
        print(ii)
        item = {'title': merged_df.iloc[ii]['sysrev']['Title'],
                'authors': merged_df.iloc[ii]['sysrev']['Authors']}
        #print(item)
        merged_df.loc[ii,('crossref','lookup')] = True
        merged_df.loc[ii,('metadata','Doi')],merged_df.loc[ii,('crossref','Title')] = fetch_doi_from_crossref(item)

    #if ii > 3: break

77
"Origin of nitrogen in the English Channel and Southern Bight of the North Sea ecosystems" Duliere, Valerie; Gypens, Nathalie; Lancelot, Christiane; Luyten, Patrick; Lacroix, Genevieve
222
"Climate Change and British Wildlife Preface" Beebee, Trevor
223
"Health risks related to seafood consumption and arsenic speciation in fish and shellfish from North Sea (Southern Bight) and Acu Port area (Brazil)" Baisch, P.; Gao, Y.; Larebeke, N. V.; Baeyens, W.; Leermakers, M.; Mirlean, N.; Junior, F. M.
266
"European Seabirds Show Stable Contemporary Biogeography" Eveillard-Buchoux, Marie; Beninger, Peter G.; Chadenas, Celine; Sellier, Dominique
326
"OIL SLICK DETECTION IN THE OFFSHORE DOMAIN: EVALUATION OF POLARIZATION-DEPENDENT SAR PARAMETERS" Angelliaume, Sebastien; Dubois-Fernandez, Pascale; Jones, Cathleen E.; Holt, Benjamin; Minchew, Brent; Amri, Emna; Miegebielle, Veronique
335
"Plastic in the Oceans Risk Management: Burden on Fish and Sea ood" Weber, Hannah Sophia
347
"IMPACT OF CLIMAT

In [31]:
merged_df.to_pickle('merged_df_post_crossref.pkl')

In [3]:
merged_df = np.load('merged_df_post_crossref.pkl',allow_pickle=True)

# Finding and removing dublicates

In [4]:
# check if there are any double entries in the merged source data
def find_dublicates(df):
    dublicates = np.zeros(shape=(0,2))
    for ii,item in enumerate(df.loc[:,'Title']):
        if ii%100 == 0: print(f'{ii/len(merged_df)*100}%')
        for jj,other_item in enumerate(df.loc[:,'Title']):
            if ii != jj:
                match_quality = jellyfish.jaro_distance(item,other_item)
                if match_quality == 1:
                    #print(df.iloc[ii]['Article ID'])
                    dubs = [df.iloc[ii]['Article ID'],df.iloc[jj]['Article ID']]
                    dublicates = np.append(dublicates,dubs)
    
    return dublicates.reshape((int(len(dublicates)/2),2))

In [5]:
#without_doi = merged_df.loc[merged_df.loc[:,("metadata",'Doi')] == '']
#dubs = find_dublicates(without_doi.loc[:,'sysrev'])

dubs = find_dublicates(merged_df.loc[:,'sysrev'])

0.0%
1.3440860215053763%
2.6881720430107525%
4.032258064516129%
5.376344086021505%
6.720430107526881%
8.064516129032258%
9.408602150537634%
10.75268817204301%
12.096774193548388%
13.440860215053762%
14.78494623655914%
16.129032258064516%
17.473118279569892%
18.817204301075268%
20.161290322580644%
21.50537634408602%
22.849462365591396%
24.193548387096776%
25.537634408602152%
26.881720430107524%
28.225806451612907%
29.56989247311828%
30.913978494623656%
32.25806451612903%
33.60215053763441%
34.946236559139784%
36.29032258064516%
37.634408602150536%
38.97849462365591%
40.32258064516129%
41.66666666666667%
43.01075268817204%
44.354838709677416%
45.69892473118279%
47.043010752688176%
48.38709677419355%
49.73118279569893%
51.075268817204304%
52.41935483870967%
53.76344086021505%
55.107526881720425%
56.451612903225815%
57.795698924731184%
59.13978494623656%
60.483870967741936%
61.82795698924731%
63.17204301075269%
64.51612903225806%
65.86021505376344%
67.20430107526882%
68.54838709677419%
69.

In [7]:
for pair in dubs:
    data_pair = pd.concat(
        [merged_df[merged_df.loc[:,('sysrev','Article ID')] == pair[0]],
         merged_df[merged_df.loc[:,('sysrev','Article ID')] == pair[1]]])
    print(data_pair)

         sysrev                                               \
     Article ID                                  Article URL   
77      9363056   https://sysrev.com/p/37916/article/9363056   
6018   10865804  https://sysrev.com/p/37916/article/10865804   

                                                                        \
                                                  Title        Journal   
77    Origin of nitrogen in the English Channel and ...  Hydrobiologia   
6018  Origin of nitrogen in the English Channel and ...  Hydrobiologia   

                                                         \
                                                Authors   
77    Duliere, Valerie; Gypens, Nathalie; Lancelot, ...   
6018  Dulière, Valérie; Gypens, Nathalie; Lancelot, ...   

                                                         \
                                               Abstract   
77    Over the last decades, nutrients loading to th...   
6018  Over the last decades, nut

In [8]:
dubs_to_drop = [merged_df.index[merged_df.loc[:,('sysrev','Article ID')] == item[1]][0] for item in dubs]

In [9]:
merged_df_with_doi_without_dubs = merged_df.drop(dubs_to_drop)

# Exporting data for fetching by "paper fetcher"

In [10]:
merged_df_with_doi_without_dubs.to_pickle('merged_df_with_doi_without_dubs')

# Checking consistency of "paper fetcher" and Joerans search with fetched pdfs while looking for the papers that are still required to be checked by hand

In [3]:
merged_df_with_doi_without_dubs = np.load('merged_df_with_doi_without_dubs',allow_pickle=True)
#merged_df_with_doi_without_dubs

renamed '10160419_' to '10160419'
removed "Oil Pollution in the North Sea: ..." from list (ended in list due to unclean fetching output dir)

In [6]:
len(fetched_ids) - len(merged_df_with_doi_without_dubs)

-1520

In [5]:
df = merged_df_with_doi_without_dubs
fetched_ids = [int(item[:-4]) for item in os.listdir('/scratch/local1/north_sea_review/merge_paper_fetchup_joeran')]

fetched_pdfs = df[0:0]
for ii,current_id in enumerate(fetched_ids):
    try:
        fetched_pdfs = fetched_pdfs.append(df.loc[df.loc[:,('sysrev','Article ID')] == current_id])
    except ValueError:
        print(current_id)

In [7]:
#outer merge the two DataFrames, adding an indicator column called 'Exist'
diff_df = pd.merge(df.loc[:,'sysrev'],fetched_pdfs.loc[:,'sysrev'], how='outer', indicator='Exist')

#find which rows don't exist in both DataFrames
diff_df = diff_df.loc[diff_df['Exist'] != 'both']
not_fetched_ids = diff_df

In [8]:
len(not_fetched_ids)

1530

why is this off by 10?

In [9]:
for current_id in fetched_ids:
    if len(df.loc[df.loc[:,('sysrev','Article ID')] == current_id]) != 1:
        print(current_id,len(df.loc[df.loc[:,('sysrev','Article ID')] == current_id]))

10685455 0
10685457 0
10685550 0
10685414 0
10685630 0
10768068 0
10768065 0
10685406 0
10685876 0
10685527 0


these ten are not in the database but somehow fetched. I assume this is a labeling issue? validate by hand

#### compare to joerans list

In [10]:
path_to_joerans_notes = '/scratch/local1/north_sea_review/joeran_manual_pdf_look-ups/no_doi_lookup_table_for_joeran_copy.csv'
notes = pd.read_csv(path_to_joerans_notes)

print("Joeran not found: "+str(len(notes[notes["Download"] == "N"])))
print("Joeran found: "+str(len(notes[notes["Download"] == "Y"])))
print("Joeran maybe found: "+str(len(notes[notes["Download"] == "D"])))

Joeran not found: 618
Joeran found: 390
Joeran maybe found: 78


In [11]:
#[print(item) for item in notes[notes["Download"] == "D"]["Notes"]]
# we need to check those individually. there are a few which shouldn't just be thrown into the mix

In [12]:
#### which ones are still required 

In [13]:
notes["Article ID"]

0        9363422
1        9363721
2        9364367
3        9364373
4        9364546
          ...   
1085    11578346
1086    11578361
1087    11578379
1088    11578387
1089    11578389
Name: Article ID, Length: 1090, dtype: int64

In [14]:
papers_already_looked_for_by_hand = []
for ii,item in enumerate(not_fetched_ids['Article ID']):
    #if ii>20: break
    #print(item,not_fetched_ids.iloc[ii])
    if len(notes[notes["Article ID"] == item]) == 1:
        #print(notes[notes["Article ID"] == item])
        papers_already_looked_for_by_hand.append(item)
len(papers_already_looked_for_by_hand)

683

In [15]:
1530 - 683

847

In [18]:
papers_in_need_for_manual_lookup = not_fetched_ids[~not_fetched_ids["Article ID"].isin(papers_already_looked_for_by_hand)]
papers_in_need_for_manual_lookup

### Exporting Data

In [20]:
merged_df_with_doi_without_dubs.to_csv('paper_db')

In [21]:
papers_in_need_for_manual_lookup.to_csv('papers_in_need_for_manual_lookup')

# ARCHVIED CODE

## Output for Joeran

### This output is not up to date with the final SysRev version as we added more papers after he looked for the missing ones of the old set

In [ ]:
sliced_df = without_doi.loc[:,[('sysrev','Article ID'),('sysrev','Title'),('sysrev','Authors'),('sysrev','Journal')]]
sliced_df.columns = sliced_df.columns.droplevel()
sliced_df.to_csv('sliced_df.csv',index=False)
sliced_df

,Article ID,Title,Authors,Journal
222,9363422,Climate Change and British Wildlife Preface,"Beebee, Trevor",NaN
335,9363721,Plastic in the Oceans Risk Management: Burden ...,"Weber, Hannah Sophia",Deutsche Lebensmittel-Rundschau
596,9364367,An illustration of the problem Offshore petrol...,"Platjouw, Froukje Maria",NaN
599,9364373,Applied Fisheries Reproductive Biology: Contri...,"Kjesbu, Olav S.",NaN
691,9364546,North Sea Region Climate Change Assessment Ove...,NaN,NaN
...,...,...,...,...
6567,11578346,Bestimmung von künstlichen Radionukliden in Ko...,"Krüger, Alois; Kanisch, Günter; Nagel, Gunther",Determination of radionuclides in fish for hum...
6570,11578361,Umschwung in der Nordseefischerei,"Hempel, Gotthilf",NaN
6576,11578379,Einfluß von Asbestabfallprodukten auf die Mies...,"Halsband, E.",NaN
6584,11578387,Kontamination der Kliesche (Limanda limanda) u...,"Büther, H.",NaN


In [5]:
sliced_df = merged_df_with_doi_without_dubs.loc[:,[('sysrev','Article ID'),('sysrev','Title'),('sysrev','Authors'),('sysrev','Journal'),('metadata','Doi')]]
sliced_df.columns = sliced_df.columns.droplevel()
sliced_df.rename(columns = {'Doi':'DOI'}, inplace = True)
sliced_df.to_csv('sliced_df.csv',index=False)
sliced_df

,Article ID,Title,Authors,Journal,DOI
0,9362819,Contemporary monitoring of storm surge activity,"Ji, Tao; Li, Guosheng",Progress in Physical Geography-Earth and Envir...,10.1177/0309133319879324
1,9362821,Reconstruction of Atlantic herring (Clupea har...,"Estrella-Martinez, Juan; Schone, Bernd R.; Thu...",Fish and Fisheries,10.1111/faf.12362
2,9362825,Modelling and predicting habitats for the neob...,"Schwemmer, Philipp; Adler, Sven; Enners, Leoni...",Estuarine Coastal and Shelf Science,10.1016/j.ecss.2019.106440
3,9362827,Multi-residue quantification and screening of ...,"Vanryckeghem, Francis; Huysman, Steve; Van Lan...",Marine Pollution Bulletin,10.1016/j.marpolbul.2019.03.049
4,9362828,Molecular analysis of Ulva compressa (Chloroph...,"Steinhagen, S.; Weinberger, F.; Karez, R.",European Journal of Phycology,10.1080/09670262.2018.1513167
...,...,...,...,...,...
7435,12028517,The lifespan of ingested plastic particles in ...,"Ryan, P. G.; Jackson, S.",Marine Pollution Bulletin,10.1007/s10113-013-0524-5
7436,12028518,Overview of the biological effects of lost and...,"Laist, D. W.",Marine Pollution Bulletin,10.3723/175605402783219181
7437,12028523,Ecological history of the Wadden Sea,"Lotze, H. K.; Reise, K.",Helgoland Marine Research,10.1016/0077-7579(91)90005-L
7438,12028525,Plastic ingestion in the North Atlantic fulmar,"van Franeker, J. A.",Marine Pollution Bulletin,10.1080/10643389.2019.1631990


# testing data

In [195]:
def print_ratio(a,b):
    print(len(a),len(b),len(a)/len(b))

In [196]:
print_ratio( merged_df.loc[merged_df.loc[:,('matching','match quality')] == 1],merged_df)

13581 14385 0.9441084462982273


In [197]:
print_ratio( merged_df.loc[ np.array(merged_df.loc[:,('metadata','Doi')] == '') + np.array(merged_df.loc[:,('matching','match quality')] != 1) ],merged_df)

3600 14385 0.2502606882168926


In [120]:
for item in set(metadata_df.Database):
    print(item)
    print(metadata_df.loc[metadata_df.Database == item,'Authors'])

GoogleScholar
index
18506            [Nieweg, D. C., Post, J. NJ, Vink, R. J.]
18507               [Sluys, R., Faubel, A., Rajagopal, S.]
18508        [Lown, A. E., Hepburn, L. J., Heywood, J. L.]
18509                                 [Lescrauwaet, A. K.]
18510    [Fockedey, N., Vandepitte, L., Ginderdeuren, K...
                               ...                        
19066                                      [Martin, D. E.]
19067      [Warmenhoven, J. P., Duiser, J. A., Leu, LT De]
19068                                      [Dubais, B. A.]
19069                                         [Mendes, B.]
19070    [Bluston, H., Consultancy, B. E., Bouchez, L. ...
Name: Authors, Length: 565, dtype: object
WoS
index
10345    [Macovei, VA, Voynova, YG, Becker, M, Triest, ...
10346    [Mueller, P, Do, HT, Smit, C, Reisdorff, C, Je...
10347    [Rishworth, GM, Adams, JB, Bird, MS, Carrasco,...
10348    [Ikpewe, IE, Baudron, AR, Ponchon, A, Fernande...
10349    [Suman, KH, Haque, MN, Uddin, MJ, 